In [1]:
import io

import netaddr

import pandas as pd

import aiohttp

from rpki_analysis.delegated_stats import read_delegated_stats, read_delegated_extended_stats

In [2]:
async with aiohttp.ClientSession() as session:
    async with session.get('https://ftp.ripe.net/pub/stats/ripencc/nro-stats/latest/nro-delegated-stats') as resp:
        df_extended = read_delegated_extended_stats(io.StringIO(await resp.text()))

In [3]:
def to_cidrs(res: netaddr.IPRange | netaddr.IPNetwork):
    if isinstance(res, netaddr.IPRange):
        return list([str(x) for x in res.cidrs()])
    elif isinstance(res, netaddr.IPNetwork):
            return [str(res)]
    else:
        return [res]

In [4]:
df_extended['resource'] = df_extended.resource.map(to_cidrs)
df_extended = df_extended.explode("resource")

In [5]:
df_extended.to_parquet('nro-delegated-stats-20241105.parquet')